In [1]:
!pip install spacy


In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00


In [5]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=ef988e5d52b43a328b253be7bc937e4b61d62c8578c39a31f832b1e33e45aba9
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [37]:
from deep_translator import GoogleTranslator
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

cntry = ""

def detect_language(text):
    try:
        lang_code = detect(text)
        return lang_code
    except LangDetectException:
        return None

def get_country_from_lang(lang_code):
    lang_country_map = {
        'hi': 'India',
        'es': 'Spain',          # Spanish
        'fr': 'France',         # French
        'de': 'Germany',        # German
        'it': 'Italy',          # Italian
        'ru': 'Russia',         # Russian
        'ja': 'Japan',          # Japanese
        'ko': 'South Korea',    # Korean
        'zh': 'China',          # Chinese
        'ur': 'Pakistan',       # Urdu
        'ar': 'Saudi Arabia',
        'dz' : 'Bhutan',
    }
    return lang_country_map.get(lang_code, 'Unknown')

def translate_and_map(input_file, output_file):
    global cntry
    with open(input_file, 'r', encoding='utf-8') as f:
        input_text = f.read()

    detected_lang = detect_language(input_text)
    if detected_lang:
        country = get_country_from_lang(detected_lang)
        cntry = country
        print(f"Detected language: {detected_lang}, Spoken most in: {country}")
        translated_text = GoogleTranslator(source=detected_lang, target='en').translate(input_text)
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(translated_text)
        print("Translation completed and stored in", output_file)
    else:
        print("It is English.")

input_file = '/content/hindi_maldives.txt'
output_file = 'translated.txt'
translate_and_map(input_file, output_file)


Detected language: hi, Spoken most in: India
Translation completed and stored in translated.txt
India


In [9]:
!pip install folium

In [11]:
!pip install geopy

In [38]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def read_text_from_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        return text
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return None


def extract_locations(text):
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    return locations

def decide_relevant_location(locations):
    if locations:
        location_counts = Counter(locations)
        most_common_location = location_counts.most_common(1)[0][0]
        return most_common_location
    else:
        return None

def process_files(ann_file_path, txt_file_path):

    text = read_text_from_file(ann_file_path)
    if text:
        extracted_locations = extract_locations(text)
        relevant_location = decide_relevant_location(extracted_locations)
        if relevant_location:
            return relevant_location

    text = read_text_from_file(txt_file_path)
    if text:
        extracted_locations = extract_locations(text)
        relevant_location = decide_relevant_location(extracted_locations)
        return relevant_location
    return None

ann_file_path = '/content/translated.txt'
txt_file_path = '/content/translated.txt'
contextual_location = process_files(ann_file_path, txt_file_path)

if contextual_location:
    print("Most relevant location:", contextual_location)
else:
    print("No relevant location found in either file.")

Most relevant location: Maldives


In [39]:
import folium
from geopy.geocoders import Nominatim

def get_coordinates(location):
    geolocator = Nominatim(user_agent="location_plotter")
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.latitude, location_info.longitude
    except Exception as e:
        print(f"Error getting coordinates for {location}: {e}")
    return None, None

def plot_locations_on_map(locations):
    first_location = get_coordinates(locations[0]) if locations else (0, 0)
    map_center = [first_location[0] or 0, first_location[1] or 0]
    map_object = folium.Map(location=map_center, zoom_start=2)

    for location in locations:
        latitude, longitude = get_coordinates(location)
        if latitude and longitude:
            folium.Marker([latitude, longitude], popup=location).add_to(map_object)

    return map_object

# print(contextual_location)
# print(cntry)
world_map = plot_locations_on_map([contextual_location,cntry])

world_map.save('location_map.html')

Maldives
India
